# 1. Lectura del dataset con pandas

In [1]:
import pandas as pd
import os

In [2]:
carpeta = 'dataset_historico_id_3195'
lista_data = os.listdir(carpeta)

for data in lista_data : # Este bucle hay que actualizarlo con los últimos días que se descargan de la API
    if data.endswith('.jsonl'):
        fichero = data

ruta = os.path.join(carpeta, fichero)
df = pd.read_json(ruta, lines = True)
df

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,presMax,horaPresMax,presMin,horaPresMin,sol
0,2024-10-05,3195,"MADRID, RETIRO",MADRID,667,"19,1","0,0","12,8",06:00,"25,4",...,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-10-06,3195,"MADRID, RETIRO",MADRID,667,"20,1","0,0","17,6",05:00,"22,6",...,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-10-07,3195,"MADRID, RETIRO",MADRID,667,"18,2","6,5","13,8",18:50,"22,5",...,81.0,97.0,Varias,43.0,12:10,NaN,NaN,NaN,NaN,NaN
3,2024-10-08,3195,"MADRID, RETIRO",MADRID,667,"15,0","0,4","12,7",05:00,"17,4",...,85.0,96.0,01:10,73.0,12:50,NaN,NaN,NaN,NaN,NaN
4,2024-10-09,3195,"MADRID, RETIRO",MADRID,667,"16,6","4,7","13,8",23:59,"19,5",...,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38406,1920-03-09,3195,"MADRID, RETIRO",MADRID,667,"1,8",Ip,"-0,7",06:00,"4,4",...,46.0,NaN,NaN,NaN,NaN,"951,7",23,"945,7",03,"8,2"
38407,1920-03-10,3195,"MADRID, RETIRO",MADRID,667,"3,5","0,0","-0,6",03:00,"7,6",...,55.0,NaN,NaN,NaN,NaN,"951,5",00,"949,1",14,"6,7"
38408,1920-03-11,3195,"MADRID, RETIRO",MADRID,667,"5,4","0,0","-1,8",05:00,"12,7",...,57.0,NaN,NaN,NaN,NaN,"951,2",09,"948,6",16,"10,0"
38409,1920-03-12,3195,"MADRID, RETIRO",MADRID,667,"10,5","0,0","5,9",00:15,"15,1",...,NaN,NaN,NaN,NaN,NaN,"949,5",10,"947,3",24,"5,7"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38411 entries, 0 to 38410
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fecha        38411 non-null  object 
 1   indicativo   38411 non-null  int64  
 2   nombre       38411 non-null  object 
 3   provincia    38411 non-null  object 
 4   altitud      38411 non-null  int64  
 5   tmed         38275 non-null  object 
 6   prec         37446 non-null  object 
 7   tmin         38275 non-null  object 
 8   horatmin     37391 non-null  object 
 9   tmax         38277 non-null  object 
 10  horatmax     37425 non-null  object 
 11  dir          29913 non-null  float64
 12  velmedia     32983 non-null  object 
 13  racha        30004 non-null  object 
 14  horaracha    29631 non-null  object 
 15  hrMedia      37038 non-null  float64
 16  hrMax        5952 non-null   float64
 17  horaHrMax    5952 non-null   object 
 18  hrMin        5951 non-null   float64
 19  hora

# 2. Pre-procesamiento

Los campos indicativo, nombre, provincia y altitud son los mismos para todos los registros, por lo que no es necesario incluirlos en el dataframe.m

In [4]:
df = df[df.columns[5:].insert(0, 'fecha')] # Se filtra el dataframe para que contenga solo las columnas de interés

Se pone en formato fecha la columna de fechas, que va a ser el campo índice

In [5]:
df["fecha"] = pd.to_datetime(df["fecha"])
df.set_index("fecha", inplace=True)
df.sort_index(inplace=True, ascending=False)

C:\Users\felix\AppData\Local\Temp\ipykernel_7828\2099670388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fecha"] = pd.to_datetime(df["fecha"])
C:\Users\felix\AppData\Local\Temp\ipykernel_7828\2099670388.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_index(inplace=True, ascending=False)


Hay campos numericos que pandas lee como texto debido a que el decimal es una coma y no un punto. Por lo tanto, se sustituye las comas por puntos

In [6]:
df.replace(',','.', regex=True, inplace=True)
columnas_numericas = ['tmed', 'prec', 'tmin', 'tmax', 'velmedia', 'racha', 'presMax', 'presMin', 'sol']
for col in columnas_numericas:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 38411 entries, 2025-04-03 to 1920-01-01
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   tmed         38275 non-null  float64
 1   prec         36038 non-null  float64
 2   tmin         38275 non-null  float64
 3   horatmin     37391 non-null  object 
 4   tmax         38277 non-null  float64
 5   horatmax     37425 non-null  object 
 6   dir          29913 non-null  float64
 7   velmedia     32983 non-null  float64
 8   racha        30004 non-null  float64
 9   horaracha    29631 non-null  object 
 10  hrMedia      37038 non-null  float64
 11  hrMax        5952 non-null   float64
 12  horaHrMax    5952 non-null   object 
 13  hrMin        5951 non-null   float64
 14  horaHrMin    5950 non-null   object 
 15  presMax      36948 non-null  float64
 16  horaPresMax  36948 non-null  object 
 17  presMin      36948 non-null  float64
 18  horaPresMin  36946 non-null  

C:\Users\felix\AppData\Local\Temp\ipykernel_7828\2596919497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(',','.', regex=True, inplace=True)
C:\Users\felix\AppData\Local\Temp\ipykernel_7828\2596919497.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')


# Hay que convertir las horas a formato numérico. Después, hacer mapa de correlación y seguir los pasos de pre-procesamiento de este notebook

https://colab.research.google.com/drive/1i4mmucD_hlHC0oT7RPoGBS5ixfRjvi9B